In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import getpass
%matplotlib inline
!pip install --upgrade bottleneck

Defaulting to user installation because normal site-packages is not writeable


In [2]:
csv_path = r"M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/final/football_academy_cleaned.csv"

df = pd.read_csv(csv_path)
df.head()

print("Loaded dataframe shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head(10))

Loaded dataframe shape: (960, 32)
Columns: ['match_id', 'match_date', 'player_id', 'player_name', 'age', 'position', 'attendance', 'minutes_played', 'fitness_score', 'stamina', 'speed', 'passing_accuracy', 'tackles', 'goals', 'assists', 'shots_on_target', 'saves', 'yellow_card', 'red_card', 'age_group', 'matches_played', 'total_minutes', 'avg_fitness', 'avg_stamina', 'avg_speed', 'avg_passing', 'total_goals', 'total_assists', 'total_saves', 'total_tackles', 'yellow_cards', 'red_cards']


,match_id,match_date,player_id,player_name,age,position,attendance,minutes_played,fitness_score,stamina,...,avg_fitness,avg_stamina,avg_speed,avg_passing,total_goals,total_assists,total_saves,total_tackles,yellow_cards,red_cards
0,M200,01-01-2025,P1000,Anvay,12,MID,1,67.0,67.3,62.1,...,66.64,59.56,65.36,70.34,2,0,0,22,3,0
1,M200,01-01-2025,P1003,Alqama,12,DEF,1,77.0,69.4,40.2,...,68.91,62.12,66.78,58.06,0,0,0,40,1,0
2,M200,01-01-2025,P1009,Amir dalvi,12,DEF,1,20.0,79.8,60.0,...,70.77,63.05,68.60,66.79,0,0,0,20,3,0
3,M200,01-01-2025,P1014,Dhruvang,12,GK,1,90.0,82.9,52.0,...,72.42,64.92,68.35,70.82,0,0,39,0,0,0
4,M200,01-01-2025,P1019,Tameem Umalkar,12,MID,1,68.0,71.1,68.2,...,70.34,58.92,68.76,69.49,0,1,0,13,1,0
5,M200,01-01-2025,P1037,Arham Tambe,12,DEF,1,89.0,71.1,52.7,...,70.58,58.20,65.17,63.91,0,0,0,40,1,0
6,M200,01-01-2025,P1038,Abdul Raafe,12,FWD,1,89.0,64.1,51.6,...,72.14,63.99,68.09,60.94,3,1,0,8,2,0
7,M200,01-01-2025,P1045,Umar,12,DEF,1,49.0,69.5,37.0,...,73.45,57.33,67.14,66.39,0,0,0,36,3,0
8,M200,01-01-2025,P1046,Hassan Arai,12,MID,1,65.0,64.7,63.7,...,70.59,60.53,64.35,72.54,0,2,0,8,1,0
9,M201,08-01-2025,P1000,Anvay,12,MID,1,62.0,73.9,70.4,...,66.64,59.56,65.36,70.34,2,0,0,22,3,0


In [3]:
display(Markdown("# 📊 Data Cleaning & PostgreSQL Export Notebook\n"))
display(Markdown("**File loaded:** `{}` | **Rows:** {:,} | **Columns:** {}".format(csv_path, df.shape[0], df.shape[1])))

# 📊 Data Cleaning & PostgreSQL Export Notebook


**File loaded:** `M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/final/football_academy_cleaned.csv` | **Rows:** 960 | **Columns:** 32

In [4]:
# Convert match_date with specific format
df['match_date'] = pd.to_datetime(df['match_date'], format="%d-%m-%Y").dt.date

# Convert to appropriate data types
int_columns = [
    'age','attendance','minutes_played','tackles','goals',
    'assists','shots_on_target','saves','yellow_card','red_card','matches_played',
    'total_minutes','total_goals','total_assists','total_saves','total_tackles',
    'yellow_cards','red_cards'
]

float_columns = ['fitness_score','stamina','speed','passing_accuracy',
                 'avg_fitness','avg_stamina','avg_speed','avg_passing']

category_columns = ['position','age_group']

string_columns = ['player_name','match_id','player_id']

# Assign types
df[int_columns] = df[int_columns].astype('int64')
df[float_columns] = df[float_columns].astype('float64')
df[category_columns] = df[category_columns].astype('category')
df[string_columns] = df[string_columns].astype('string')

In [5]:
df.info() 
df.describe(include='all')
df.isna().sum()
print(df.isna().sum())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   match_id          960 non-null    string  
 1   match_date        960 non-null    object  
 2   player_id         960 non-null    string  
 3   player_name       960 non-null    string  
 4   age               960 non-null    int64   
 5   position          960 non-null    category
 6   attendance        960 non-null    int64   
 7   minutes_played    960 non-null    int64   
 8   fitness_score     960 non-null    float64 
 9   stamina           960 non-null    float64 
 10  speed             960 non-null    float64 
 11  passing_accuracy  960 non-null    float64 
 12  tackles           960 non-null    int64   
 13  goals             960 non-null    int64   
 14  assists           960 non-null    int64   
 15  shots_on_target   960 non-null    int64   
 16  saves             960 non-

,match_id,match_date,player_id,player_name,age,position,attendance,minutes_played,fitness_score,stamina,...,avg_fitness,avg_stamina,avg_speed,avg_passing,total_goals,total_assists,total_saves,total_tackles,yellow_cards,red_cards
0,M200,2025-01-01,P1000,Anvay,12,MID,1,67,67.3,62.1,...,66.64,59.56,65.36,70.34,2,0,0,22,3,0
1,M200,2025-01-01,P1003,Alqama,12,DEF,1,77,69.4,40.2,...,68.91,62.12,66.78,58.06,0,0,0,40,1,0
2,M200,2025-01-01,P1009,Amir dalvi,12,DEF,1,20,79.8,60.0,...,70.77,63.05,68.60,66.79,0,0,0,20,3,0
3,M200,2025-01-01,P1014,Dhruvang,12,GK,1,90,82.9,52.0,...,72.42,64.92,68.35,70.82,0,0,39,0,0,0
4,M200,2025-01-01,P1019,Tameem Umalkar,12,MID,1,68,71.1,68.2,...,70.34,58.92,68.76,69.49,0,1,0,13,1,0


In [15]:
df.describe()

,age,attendance,minutes_played,fitness_score,stamina,speed,passing_accuracy,tackles,goals,assists,...,avg_fitness,avg_stamina,avg_speed,avg_passing,total_goals,total_assists,total_saves,total_tackles,yellow_cards,red_cards
count,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,...,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000
mean,14.791667,0.922917,64.276042,69.838958,60.136875,63.737917,66.767500,1.071875,0.033333,0.034375,...,69.841458,60.139167,63.740625,66.769583,0.666667,0.687500,2.895833,21.437500,1.416667,0.020833
std,1.937053,0.266863,25.100712,9.940482,10.648750,10.460888,12.764259,1.296518,0.185314,0.187919,...,1.673314,2.621237,2.902458,3.732569,1.143205,1.102774,9.679407,13.874014,1.239485,0.142901
min,12.000000,0.000000,0.000000,35.800000,28.800000,32.400000,23.300000,0.000000,0.000000,0.000000,...,66.350000,52.450000,57.330000,58.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,1.000000,53.000000,63.600000,53.200000,56.675000,58.050000,0.000000,0.000000,0.000000,...,68.592500,58.895000,61.880000,63.877500,0.000000,0.000000,0.000000,11.750000,0.000000,0.000000
50%,15.000000,1.000000,69.500000,70.050000,59.500000,63.900000,67.100000,1.000000,0.000000,0.000000,...,70.070000,60.055000,63.755000,66.795000,0.000000,0.000000,0.000000,17.000000,1.000000,0.000000
75%,16.000000,1.000000,86.000000,76.400000,67.100000,70.625000,75.300000,2.000000,0.000000,0.000000,...,70.950000,61.927500,65.407500,69.712500,1.000000,1.000000,0.000000,36.000000,2.000000,0.000000
max,18.000000,1.000000,90.000000,97.800000,96.900000,94.400000,100.000000,7.000000,2.000000,2.000000,...,73.450000,64.920000,69.270000,73.390000,4.000000,4.000000,39.000000,45.000000,5.000000,1.000000


In [7]:
from IPython.display import display, Markdown

display(Markdown('### Outlier Check (IQR Method)'))

num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outlier_mask = (df[num_cols] < lower_bound) | (df[num_cols] > upper_bound)

outliers = df[outlier_mask.any(axis=1)]

print(f"Total Outlier Rows Detected: {outliers.shape[0]}\n")
display(outliers.head(10))

### Outlier Check (IQR Method)

Total Outlier Rows Detected: 505



,match_id,match_date,player_id,player_name,age,position,attendance,minutes_played,fitness_score,stamina,...,avg_fitness,avg_stamina,avg_speed,avg_passing,total_goals,total_assists,total_saves,total_tackles,yellow_cards,red_cards
1,M200,2025-01-01,P1003,Alqama,12,DEF,1,77,69.4,40.2,...,68.91,62.12,66.78,58.06,0,0,0,40,1,0
3,M200,2025-01-01,P1014,Dhruvang,12,GK,1,90,82.9,52.0,...,72.42,64.92,68.35,70.82,0,0,39,0,0,0
6,M200,2025-01-01,P1038,Abdul Raafe,12,FWD,1,89,64.1,51.6,...,72.14,63.99,68.09,60.94,3,1,0,8,2,0
12,M201,2025-01-08,P1014,Dhruvang,12,GK,1,90,64.0,63.9,...,72.42,64.92,68.35,70.82,0,0,39,0,0,0
15,M201,2025-01-08,P1038,Abdul Raafe,12,FWD,1,71,74.1,83.3,...,72.14,63.99,68.09,60.94,3,1,0,8,2,0
18,M202,2025-01-15,P1000,Anvay,12,MID,1,73,77.7,57.8,...,66.64,59.56,65.36,70.34,2,0,0,22,3,0
20,M202,2025-01-15,P1009,Amir dalvi,12,DEF,1,90,78.9,69.9,...,70.77,63.05,68.60,66.79,0,0,0,20,3,0
21,M202,2025-01-15,P1014,Dhruvang,12,GK,1,90,75.3,66.0,...,72.42,64.92,68.35,70.82,0,0,39,0,0,0
22,M202,2025-01-15,P1019,Tameem Umalkar,12,MID,1,90,76.5,48.9,...,70.34,58.92,68.76,69.49,0,1,0,13,1,0
24,M202,2025-01-15,P1038,Abdul Raafe,12,FWD,1,66,69.5,57.3,...,72.14,63.99,68.09,60.94,3,1,0,8,2,0


In [8]:
display(Markdown('## 📈 KPIs & Automated Insights'))

# Defining categorical and numerical columns 
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

kpi_items = []
# Total records
kpi_items.append(("Total records", df.shape[0]))
# Unique values for the top 5 categorical columns
for c in cat_cols[:5]:
    kpi_items.append((f"Unique ({c})", df[c].nunique()))
# Numeric summary for top 5 numeric columns
num_summary = df[num_cols].describe().T.head(5)

display(Markdown('### KPI Summary'))
for k, v in kpi_items:
    display(Markdown(f"- **{k}:** {v}"))
display(Markdown('### Top numeric column summary'))
display(num_summary)

## 📈 KPIs & Automated Insights

### KPI Summary

- **Total records:** 960

- **Unique (match_date):** 20

- **Unique (position):** 4

- **Unique (age_group):** 4

### Top numeric column summary

,count,mean,std,min,25%,50%,75%,max
age,960.0,14.791667,1.937053,12.0,13.0,15.00,16.0,18.0
attendance,960.0,0.922917,0.266863,0.0,1.0,1.00,1.0,1.0
minutes_played,960.0,64.276042,25.100712,0.0,53.0,69.50,86.0,90.0
fitness_score,960.0,69.838958,9.940482,35.8,63.6,70.05,76.4,97.8
stamina,960.0,60.136875,10.648750,28.8,53.2,59.50,67.1,96.9


In [9]:
output_path = r"M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/final/football_academy_cleaned_safe.csv"
df.to_csv(output_path, index=False)
print("Cleaned CSV exported to:", output_path)

Cleaned CSV exported to: M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/final/football_academy_cleaned_safe.csv


In [10]:
!pip install sqlalchemy psycopg2-binary pandas

Defaulting to user installation because normal site-packages is not writeable


In [14]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

# Step 1: Connecting to PostgreSQL
username = "postgres"      
password = "987021" 
host = "localhost"         
port = "5432"              
database = "FA_db"    # the database created in pgAdmin

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# Step 2: Loading DataFrame into PostgreSQL
table_name = "FA_P&E_TAB"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'FA_P&E_TAB' in database 'FA_db'.
